In [1]:
import pandas as pd
import random
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import csv
from sklearn.cluster import DBSCAN
import math
import statistics
pd.options.mode.chained_assignment = None  # default='warn'
import itertools

#Convert CSV to Dataframe
data = pd.read_csv("./data/iou-dsc.csv", header=[0])
data["FOLDER"] = data["IN FILE"].apply(lambda x: "")
data

,IN FILE,CLIP LENGTH,OFFSET,DURATION,MAX FREQ,MIN FREQ,SAMPLE RATE,MANUAL ID,TIME_SPENT,LAST MOD BY,LAST MOD DATE,LAST MOD TIME,FOLDER
0,XC425555_-_Screaming_Piha_-_Lipaugus_vociferan...,15.73850,0.6511,2.1214,24000,0,44100,bird,11.652,admin,NaN,NaN,
1,XC425555_-_Screaming_Piha_-_Lipaugus_vociferan...,15.73850,12.6092,3.1042,24000,0,44100,bird,11.654,admin,NaN,NaN,
2,XC425555_-_Screaming_Piha_-_Lipaugus_vociferan...,15.73850,14.1490,1.3842,24000,0,44100,bird,33.147,nelly,NaN,NaN,
3,XC425555_-_Screaming_Piha_-_Lipaugus_vociferan...,15.73850,1.1590,1.3760,24000,0,44100,bird,33.150,nelly,NaN,NaN,
4,XC425555_-_Screaming_Piha_-_Lipaugus_vociferan...,15.73850,1.1622,0.6098,24000,0,44100,bird,166.323,edmundo,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,XC591821_-_Screaming_Piha_-_Lipaugus_vociferan...,5.66646,4.3832,0.0527,24000,0,44100,bird,95.519,edmundo,NaN,NaN,
73,XC591821_-_Screaming_Piha_-_Lipaugus_vociferan...,5.66646,3.2886,0.5116,24000,0,44100,bird,95.504,edmundo,NaN,NaN,
74,XC591821_-_Screaming_Piha_-_Lipaugus_vociferan...,5.66646,3.7851,0.5643,24000,0,44100,bird,95.522,edmundo,NaN,NaN,
75,XC591821_-_Screaming_Piha_-_Lipaugus_vociferan...,5.66646,2.7602,1.7457,24000,0,44100,bird,57.017,ryan,NaN,NaN,


In [27]:
def is_overlap(offset_df, end_df, chunk_start, chunk_end):
    is_both_before = (chunk_end < offset_df) & (chunk_start < offset_df)
    is_both_after = (end_df < chunk_end) & (end_df < chunk_start)
    return (~is_both_before) & (~is_both_after)

def annotation_chunker(kaleidoscope_df, chunk_length):
    """
    Function that converts a Kaleidoscope-formatted Dataframe containing 
    annotations to uniform chunks of chunk_length.

    Note: if all or part of an annotation covers the last < chunk_length
    seconds of a clip it will be ignored. If two annotations overlap in 
    the same 3 second chunk, both are represented in that chunk

    Args:
        kaleidoscope_df (Dataframe)
            - Dataframe of annotations in kaleidoscope format

        chunk_length (int)
            - duration to set all annotation chunks
    Returns:
        Dataframe of labels with chunk_length duration 
        (elements in "OFFSET" are divisible by chunk_length).
    """

    #Init list of clips to cycle through and output dataframe
    kaleidoscope_df["FILEPATH"] =  kaleidoscope_df["FOLDER"] + kaleidoscope_df["IN FILE"] 
    clips = kaleidoscope_df["FILEPATH"].unique()
    df_columns = {'FOLDER': 'str', 'IN FILE' :'str', 'CLIP LENGTH' : 'float64', 'CHANNEL' : 'int64', 'OFFSET' : 'float64',
                'DURATION' : 'float64', 'SAMPLE RATE' : 'int64','MANUAL ID' : 'str'}
    output_df = pd.DataFrame({c: pd.Series(dtype=t) for c, t in df_columns.items()})
    
    # going through each clip
    for clip in clips:
        clip_df = kaleidoscope_df[kaleidoscope_df["FILEPATH"] == clip]
        path = clip_df["FOLDER"].unique()[0]
        file = clip_df["IN FILE"].unique()[0]
        birds = clip_df["MANUAL ID"].unique()
        sr = clip_df["SAMPLE RATE"].unique()[0]
        clip_len = clip_df["CLIP LENGTH"].unique()[0]

        # quick data sanitization to remove very short clips
        # do not consider any chunk that is less than chunk_length
        if clip_len < chunk_length:
            continue
        potential_annotation_count = int(clip_len)//int(chunk_length)

        # going through each species that was ID'ed in the clip
        arr_len = int(clip_len*1000)
        for bird in birds:
            species_df = clip_df[clip_df["MANUAL ID"] == bird]
            human_arr = np.zeros((arr_len))
            # looping through each annotation
            for annotation in species_df.index:
                minval = int(round(species_df["OFFSET"][annotation] * 1000, 0))
                # Determining the end of a human label
                maxval = int(
                    round(
                        (species_df["OFFSET"][annotation] +
                         species_df["DURATION"][annotation]) *
                        1000,
                        0))
                # Placing the label relative to the clip
                human_arr[minval:maxval] = 1
            # performing the chunk isolation technique on the human array

            for index in range(potential_annotation_count):
                chunk_start = index * (chunk_length*1000)
                chunk_end = min((index+1)*chunk_length*1000,arr_len)
                chunk = human_arr[int(chunk_start):int(chunk_end)]
                if max(chunk) >= 0.5:
                    #Get row data
                    row = pd.DataFrame(index = [0])
                    annotation_start = chunk_start / 1000

                    overlap = is_overlap(clip_df["OFFSET"], clip_df["OFFSET"] + clip_df["DURATION"], annotation_start, annotation_start + chunk_length)
                    annotation_df = clip_df[overlap]
                    
                    for i in range(annotation_df.shape[0]):
                        temp = annotation_df.iloc[i]
                        #updating the dictionary
                        #if ('CONFIDENCE' in temp.rows):
                        #    row["CONFIDENCE"] = max(temp["CONFIDENCE"])
                        #else:
                        #    row["CONFIDENCE"] = 0
                        row["FOLDER"] = path
                        row["IN FILE"] = file
                        row["CLIP LENGTH"] = clip_len
                        row["OFFSET"] = annotation_start
                        row["DURATION"] = chunk_length
                        row["SAMPLE RATE"] = sr
                        row["MANUAL ID"] = bird
                        row["CHANNEL"] = 0
                        row["LAST MOD BY"] = temp["LAST MOD BY"]
                        output_df = pd.concat([output_df,row], ignore_index=True)
    return output_df

In [69]:
def majority_vote(df, users, chunk_length=1):
    iou_scores = np.array([])
    df = df[df["LAST MOD BY"].isin(users)]
    df = annotation_chunker(df, chunk_length)
    df = df.groupby(by=["LAST MOD BY", "OFFSET"]).max().reset_index()
    #TODO ADD SPECIES STUFF HERE
    df = df.groupby(by=["OFFSET"]).count().reset_index()
    counts = (df["FOLDER"]/len(users)).mean() #Does mean make sense here?
    return counts

In [70]:
data.iloc[0]["IN FILE"]

'XC425555_-_Screaming_Piha_-_Lipaugus_vociferans.mp3'

In [71]:
df = data[data["IN FILE"] == 'XC425555_-_Screaming_Piha_-_Lipaugus_vociferans.mp3']
users = df["LAST MOD BY"].unique()
majority_vote(df, users)

0.7291666666666667